# Import library

In [113]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

### Avoid difference result after rerun notebook

In [114]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Reading dataset

In [115]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')

In [116]:
len(no_dir)

535

In [117]:
len(yes_dir)

557

In [118]:
data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)

In [119]:
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)

In [120]:
data_set = np.array(data_set)
label = np.array(label)
data_set.shape

(1092, 64, 64, 3)

In [121]:
label.shape

(1092,)

In [122]:
print(f'yes observe:{sum(label)}, no observe:{len(label)-sum(label)}')

yes observe:557, no observe:535


# Split and normalize data


## 1. Split


In [123]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=99
    )
x_train,x_val,y_train,y_val = train_test_split(
        x_train,y_train,
    test_size=0.25,
    random_state=99
)


In [124]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape}\nX validation shape: {x_val.shape}\nY validation shape: {x_val.shape}')

X train shape: (654, 64, 64, 3)
Y train shape: (654,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,)
X validation shape: (219, 64, 64, 3)
Y validation shape: (219, 64, 64, 3)


## 2. Normalize

In [125]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)
x_val = normalize(x_val, axis=1)

# Building Model

In [126]:
from keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Activation,
    Dropout,
    Flatten,
    Dense
)

In [127]:
model=Sequential()
INPUT_SIZE = 64

model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(1))
model.add(Activation('sigmoid'))

e:\Anaconda\Anaconda setup\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training model

In [128]:
model.compile(
    loss='binary_crossentropy',
    # Set amsgrad to True for reproducibility
    optimizer=Adam(amsgrad=True), 
    metrics=['accuracy']
)
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    verbose=1,
    epochs=10,
    validation_data=(x_val,y_val),
    shuffle=False
    )

Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.5018 - loss: 0.7182 - val_accuracy: 0.6027 - val_loss: 0.6860
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5439 - loss: 0.6876 - val_accuracy: 0.7489 - val_loss: 0.6657
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6084 - loss: 0.6544 - val_accuracy: 0.7534 - val_loss: 0.6409
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6634 - loss: 0.6120 - val_accuracy: 0.7763 - val_loss: 0.5317
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6812 - loss: 0.5614 - val_accuracy: 0.8128 - val_loss: 0.4215
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7902 - loss: 0.4521 - val_accuracy: 0.8311 - val_loss: 0.3939
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8124 - loss: 0.3989 - val_accuracy: 0.8813 - val_loss: 0.3194
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8582 - loss: 0.3504 - val_accuracy: 0.8995 - v

# Test with new data

In [129]:
y_test_pred = model.predict(x_test)
print(y_test_pred.shape)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
(219, 1)


In [130]:
y_pred = (y_test_pred >0.5).astype(int)
y_pred.shape

(219, 1)

In [131]:
y_test.shape

(219,)

In [132]:
y_test_reshape = y_test.reshape(-1,1)
y_test_reshape.shape

(219, 1)

In [133]:
print("Accuracy in test set:", accuracy_score(y_test_reshape, y_pred))
print('Accuracy in validation set:',history.history['val_accuracy'][-1])

Accuracy in test set: 0.9360730593607306
Accuracy in validation set: 0.9086757898330688


-> So with base case equal to 93.9%, this model be overfit!

# Save original model 

**Infor**<br>
Name: BrainTurmor_v1<br>
Accuracy in validation set: 0.9452054500579834<br>
Accuracy in test set: 0.867579908675799<br>
Status: Overfiting

In [134]:
model.save('BrainTurmor_v1.keras')